<a href="https://colab.research.google.com/github/dotdigital26/Data_Science_Projects/blob/master/CodeAgainstHate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy
from sqlite3 import connect
from os.path import isfile
from os import access, R_OK
from sys import exit

db_file = 'tweets.db'

# More here https://apps.twitter.com
consumer_key = 'HOaIaYGFENEX89xeEf74K7igF'
consumer_secret = 'pdoFB5brCIrTnYXVYi7TNBvpJLdZuggCJNbGNoECMPEM1d12c8'
access_token = '1268078914259345409-wo0Yaqjt11o2Hpa4C5hr4Q70ZAYSoG'
access_token_secret = 'Gyewe2Vbig23budKhMy5xAFD1TQ6A1MwuP8GtXe2N2TQk'

def oauth_authenticate():
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  return tweepy.API(auth)

def db_file_exists(path):
  return isfile(path) and access(path, R_OK)

def db_create(sqlite_file):
  conn = connect(sqlite_file)
  create_table =  'CREATE TABLE tweets ('
  # Could be used for https://twitter.com/statuses/<status_id>
  create_table += 'status_id INTEGER PRIMARY KEY, '
  create_table += 'created_at DATETIME, '
  create_table += 'screen_name VARCHAR(30), '
  create_table += 'tweet VARCHAR(255)'
  create_table += ')'

  conn.cursor().execute(create_table)
  conn.commit()
  conn.close()

def db_connect(sqlite_file):
  return connect(sqlite_file)


class MyStreamListener(tweepy.StreamListener):

  def on_status(self, status):
    status_id = status.id
    created_at = status.created_at
    screen_name = status.user.screen_name
    tweet = status.text

    print(str(created_at) + ' ' + '<@' + screen_name + '> ' + tweet)

    conn = db_connect(db_file)
    query = "INSERT INTO tweets(status_id, created_at, screen_name, tweet) VALUES (?, ?, ?, ?)"
    conn.cursor().execute(query, (status_id, created_at, screen_name, tweet))
    conn.commit()
    conn.close()


if __name__ == "__main__":
  # User authentication with OAuth
  api = oauth_authenticate()

  # Create the database file, if missing
  if not db_file_exists(db_file):
    db_create(db_file)

  # Listen for the user stream
  myStreamListener = MyStreamListener()
  myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)
  try:
    myStream.userstream()
    myStream.filter(track=['BBN']) # For debugging only
  except KeyboardInterrupt:
    print('Program terminated.')
    exit(0)
  except AttributeError:
    pass